In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE146264"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE146264"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE146264.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE146264.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE146264.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE146264_family.soft.gz', 'GSE146264_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Melanoma/GSE146264/GSE146264_family.soft.gz
Matrix file: ../../input/GEO/Melanoma/GSE146264/GSE146264_series_matrix.txt.gz
Background Information:
!Series_title	"Single cell RNA-seq of psoriatic skin identifies pathogenic Tc17 subsets and reveals distinctions between CD8+ T cells in autoimmunity and cancer"
!Series_summary	"Psoriasis is an inflammatory, IL-17-driven skin disease in which autoantigen-induced CD8+ T cells have been identified as pathogenic drivers. In this study, we used single-cell RNA-seq to identify 11 transcriptionally diverse CD8+ T cell subsets in psoriatic and healthy skin. Among several inflammatory subsets enriched in psoriatic skin, we observed two Tc17 subsets that were metabolically divergent, developmentally related, and expressed CXCL13, which we found to be a biomarker of psoriasis severity. Despite high co-inhibitory receptor expression in the

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any
import json
import re

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be single-cell RNA-seq data of CD8+ T cells
# This is gene expression data, so we set is_gene_available to True
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait, looking at the sample IDs in row 1, we can see prefixes 'P' (likely psoriasis) and 'C' (likely control)
# This can be used to determine disease status
trait_row = 1  

# No age information is provided in the sample characteristics dictionary
age_row = None

# No gender information is provided in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait value to binary (0 for control, 1 for psoriasis)
    
    Args:
        value: String containing subject ID
        
    Returns:
        1 if subject has psoriasis (P prefix), 0 if control (C prefix), None otherwise
    """
    if not value or ':' not in value:
        return None
    
    # Extract the value after the colon and remove leading/trailing spaces
    subjectid = value.split(':', 1)[1].strip()
    
    # Check if the subject ID starts with 'P' (psoriasis) or 'C' (control)
    if subjectid.startswith('P'):
        return 1  # Psoriasis
    elif subjectid.startswith('C'):
        return 0  # Control
    else:
        return None

# No convert_age function needed since age_row is None

# No convert_gender function needed since gender_row is None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assume clinical_data is already loaded from a previous step
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # clinical_data from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=None,  # Not needed since age_row is None
        gender_row=gender_row,
        convert_gender=None  # Not needed since gender_row is None
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Clinical Data Preview:
{'GSM4369177': [0.0], 'GSM4369178': [0.0], 'GSM4369179': [0.0], 'GSM4369180': [0.0], 'GSM4369181': [0.0], 'GSM4369182': [0.0], 'GSM4369183': [0.0], 'GSM4369184': [0.0], 'GSM4369185': [0.0], 'GSM4369186': [0.0], 'GSM4369187': [0.0], 'GSM4369188': [0.0], 'GSM4369189': [0.0], 'GSM4369190': [0.0], 'GSM4369191': [0.0], 'GSM4369192': [0.0], 'GSM4369193': [0.0], 'GSM4369194': [0.0], 'GSM4369195': [0.0], 'GSM4369196': [1.0], 'GSM4369197': [1.0], 'GSM4369198': [1.0], 'GSM4369199': [1.0], 'GSM4369200': [1.0], 'GSM4369201': [1.0], 'GSM4369202': [1.0], 'GSM4369203': [1.0], 'GSM4369204': [1.0], 'GSM4369205': [1.0], 'GSM4369206': [1.0], 'GSM4369207': [1.0], 'GSM4369208': [1.0], 'GSM4369209': [1.0], 'GSM4369210': [1.0], 'GSM4369211': [1.0], 'GSM4369212': [1.0], 'GSM4369213': [1.0], 'GSM4369214': [1.0], 'GSM4369215': [1.0], 'GSM4369216': [1.0], 'GSM4369217': [1.0], 'GSM4369218': [1.0], 'GSM4369219': [1.0], 'GSM4369220': [1.0], 'GSM4369221': [1.0], 'GSM4369222': [1.0], 'GSM436922

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")

This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 0
First 20 gene/probe identifiers:
Index([], dtype='object', name='ID')
